In [1]:
import sys
sys.path.append('/Users/snow/Downloads/DeepLearningTutorials-master/code/')

import numpy as np
import pandas as pd
import theano

# from keras.layers import containers
from keras import models
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dropout, Activation, Dense

from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.optimizers import SGD, Adam, RMSprop

from keras.datasets import mnist
from keras.layers.recurrent import GRU

from keras import backend as K
from keras.models import model_from_json
from keras.callbacks import RemoteMonitor
from keras.datasets import mnist

# from bokeh.charts import Scatter, output_file, show
from bokeh.client import push_session
from bokeh.plotting import figure, curdoc, output_server
from bokeh.charts import Scatter, show

def load_skeleton(json_path):
    '''Load keras skeleton model
    input
    -----
    json_path: str
          json skeleton model from keras
    output
    ------
    keras model structure
    '''
    ## Load model structure saved as numypy binary
    model_json = str(np.load(json_path))

    ## Convert json string to keras model
    ae_model = model_from_json(model_json)
    return ae_model

def load_weights(keras_model, h5_path):
    '''Load weights of a saved keras model
    input
    -----
    keras_model: keras skeleton model output from load_skeleton
    h5_path: str
             path of saved keras file. .h5 file extension
    output
    ------
    keras model structure with desired weights loaded

    '''
    ## Load trained weights
    keras_model.load_weights(h5_path)
    return keras_model

def load_data(data_path):
    '''Load and normalise data
    input
    -----
    data_path: str
               file path of data
    output
    ------
    dataframe
    '''
    ## Load raw data
    main_df = pd.read_excel(data_path)
    main_df_T = main_df.T
    ## n x p
    main_df_T = main_df_T[main_df_T.columns][1:]
    ## p x n
    # main_df_T = main_df_T[main_df_T.columns[1:]]
    print('Dataset shape',main_df_T.shape)

    ## Normalise
    # main_df_T = (main_df_T - main_df_T.mean(axis=0))/main_df_T.std(axis=0) ## Z-score normalised
    ## Between 0 and 1 normalised
    main_df_01N = (main_df_T - main_df_T.min(axis=0))/(main_df_T.max(axis=0) - main_df_T.min(axis=0)) 
    return main_df_01N

def get_layer_output(keras_model, dataframe):
    '''
    input
    -----
    keras_model: keras model object
    dataframe: pandas.DataFrame

    output
    ------
    numpy.ndarray

    '''
    ## Convert python object as theano shared var
    X_train = theano.shared(np.asarray(dataframe, dtype=theano.config.floatX))

    ## Function to get the output data of the hidden layer
    keras_model.layers[0].output_reconstruction = False ## get the deepest layer
    get_1st_layer_output = K.function([keras_model.layers[0].input],
                                  [keras_model.layers[0].get_output(train=False)])
    layer_output = get_1st_layer_output([X_train.get_value()])[0]
    return layer_output

Using Theano backend.


###### 1. Load dataset and pre

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [9]:
# ## A different from kenko
# data_path = '/Users/snow/Documents/kenkodo/test_data_znorm.csv'
# X_train = pd.read_csv(data_path)

In [10]:
json_path = '/Users/snow/Documents/kenkodo/main/models/ae_kd1.npy'
model_path = '/Users/snow/Documents/kenkodo/main/models/ae_kd1.h5'
# data_path = '/Users/snow/Documents/kenkodo/test_data_znorm.csv'

## Keras skeleton model
ae_model = load_skeleton(json_path)

## Keras model with desired weights
ae_model = load_weights(ae_model, model_path)

## Load raw data
main_df_01N = pd.DataFrame(X_train)

## Get layer output
layer_output = get_layer_output(ae_model, main_df_01N)

KeyError: 'class_name'

In [ ]:
new_xtrain_2d = pd.DataFrame(layer_output)
new_xtrain_2d['label'] = y_train[:50000]
new_xtrain_2d.columns = ['x', 'y', 'label']

p = figure()
r2 = p.scatter(np.arange(100),np.arange(100), marker='circle', size=3,
              line_color="navy", fill_color="orange", alpha=0.5)
# open a session to keep our local document in sync with server
session = push_session(curdoc())

# Datasource
ds = r2.data_source

def update():
    global layer_output
    global ae_model
    ae_model = load_weights(ae_model, model_path)
    layer_output = get_layer_output(ae_model, main_df_01N)
    ds.data["x"] = layer_output[:,0]
    ds.data["y"] = layer_output[:,1]
    ds.data
    ds.trigger('data', ds.data, ds.data)
    
curdoc().add_periodic_callback(update, 50)
session.show() # open the document in a browser
session.loop_until_closed() # run forever

In [ ]:
p = figure()
r2 = p.scatter(np.arange(100),np.arange(100), marker='circle', size=3,
              line_color="navy", fill_color="orange", alpha=0.5)

In [ ]:
ds = r2.data_source
ds.data["x"] = layer_output[:,0]
ds.data["y"] = layer_output[:,1]

In [ ]:
r2.data_source.data['x'].shape

In [11]:
session = push_session(curdoc())
session.show()

OSError: Cannot push session document because we failed to connect to the server (to start the server, try the 'bokeh serve' command)

In [4]:
new_xtrain_2d = pd.DataFrame(layer_output)
new_xtrain_2d['label'] = y_train[:50000]
new_xtrain_2d.columns = ['x', 'y', 'label']
p = Scatter(new_xtrain_2d,x=new_xtrain_2d.columns[0], y=new_xtrain_2d.columns[1], color='label', marker='label', legend='top_right')


In [7]:
p = Scatter(new_xtrain_2d[:100],x=new_xtrain_2d.columns[0], y=new_xtrain_2d.columns[1], color='label', marker='label', legend='top_right')

In [6]:
session = push_session(curdoc())
session.show()